# Imports

In [1]:
from __future__ import annotations
import packages

from context.utils import typer as t
from context.infra import clients
from context.utils.handlers import print_hldr
from context.infra.clients import logger, manager_mongodb

from toolkit.utils import utils
from toolkit.utils.utils import rp_print
from toolkit.utils.llm import main as utils_llm

import context.instances as inst
import context.consts as const
import context.settings.main as settings_main

from toolkit.llm.langchain.core import integration, utils as utils_lc
from toolkit.llm.langchain.data.indexing import (
    documents, document_loaders, text_splitters,
)
from toolkit.llm.langchain.data.persistence import retrievers
from toolkit.llm.langchain.execution import (
    runnables, graphs, tools, agents, tools as tools_lc
)
from toolkit.llm.langchain.models import (
    prompts as prompts_lc, llms, messages as msgs_lc,
)

2025-03-03 09:10:41.863 | INFO     | packages:<module>:68 - apps directory: /Users/thung/Documents/Me/Coding/Embedded-AI/apps
2025-03-03 09:10:41.863 | INFO     | packages:<module>:69 - Toolkit path: /Users/thung/Documents/Me/Coding/Embedded-AI/apps/toolkit
2025-03-03 09:10:41.864 | INFO     | packages:<module>:70 - Environment files loaded:
2025-03-03 09:10:41.864 | INFO     | packages:<module>:72 -   - /Users/thung/Documents/Me/Coding/Embedded-AI/apps/ports.env
2025-03-03 09:10:41.864 | INFO     | packages:<module>:72 -   - /Users/thung/Documents/Me/Coding/Embedded-AI/apps/.env


/opt/homebrew/Caskroom/miniconda/base/envs/dev/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-03 09:10:43.202 | INFO     | toolkit.db.mongodb:connect:436 - Successfully connected to MongoDB database: app
2025-03-03 09:10:43.203 | INFO     | toolkit.db.mongodb:get_collection:447 - Creating manager for collection: vehicle
2025-03-03 09:10:43.203 | INFO     | toolkit.db.redix:connect:422 - Successfully connected to Redis database: 0
2025-03-03 09:10:43.203 | INFO     | toolkit.db.redix:get_namespace:433 - Creating manager for namespace: llm
2025-03-03 09:10:44.328 | WARNING  | context.settings.main:<module>:28 - Running in DEVELOPMENT mode
2025-03-03 09:10:44.470 | INFO     | toolkit.llm.langchain.models.llms:create_llm:78 - 🔹 LLM OpenAI gpt-4o-mini
2025-03-03 09:10:44.510 | INFO     | toolkit.llm

# Dev

## RAG

In [2]:
from services.llm.agents.vehicle.context import prompts_agents_rag

class NODE_RAG(t.EnumCustom):
	RETRIEVE = t.auto()

class DATA_RAG(t.EnumCustom):
	RETRIEVED_DATA = t.auto()

class AgentRAG(agents.AgentBase):
	def __init__(
		self, 
		name: str,
		node=NODE_RAG,
		retriever = inst.retrievers_qdrant["vehicle"]
	):
		super().__init__(name=name, llm=inst.llm_main)

		self.node = node
		self.retriever = retriever
		self.build()
					
	@utils.print_async_function_name
	async def node_retrieve(
		self, state: agents.AgentBase.State
	) -> agents.AgentBase.State:
		user_query = state['messages'][-1].content
		
		retrieved_data = await self.retriever.ainvoke(user_query)
		retrieved_data = await retrievers.extract_retriever_results(retrieved_data)
		
		return {
			"data": {
				DATA_RAG.RETRIEVED_DATA: retrieved_data,
			},
		}

	@utils.print_async_function_name
	async def node_core(
		self, state: agents.AgentBase.State
	) -> agents.AgentBase.State:
		user_query = state['messages'][-1].content
		
		prompt_tpl = prompts_lc.ChatPromptTemplate([
			("system", prompts_agents_rag["system"]),
			("human", prompts_agents_rag["generate"]),
		])

		prompt = await prompt_tpl.ainvoke({
			"user_query": user_query,
			"retrieved_data": state["data"][DATA_RAG.RETRIEVED_DATA],
		})

		result = await inst.llm_main.ainvoke(prompt)
		
		return {
			"result": result,
			"messages": msgs_lc.AIMessage(str(result)),
		}

	async def node_delete_all_messages(
		self, state: agents.AgentBase.State
	) -> agents.AgentBase.State:
		messages: list[msgs_lc.AnyMessage] = state["messages"]
		return {"messages": [msgs_lc.RemoveMessage(id=m.id) for m in messages]}
							
	def architect(self):
		self.builder.add_node(self.node.RETRIEVE, self.node_retrieve)
		self.builder.add_node(self.NODE.CORE, self.node_core)

		self.builder.add_edge(graphs.START, self.node.RETRIEVE)
		self.builder.add_edge(self.node.RETRIEVE, self.NODE.CORE)

		# self.builder.add_edge(self.NODE.CORE, graphs.END)
		self.builder.add_node(self.NODE.CLEAR_MSGS, self.node_delete_all_messages)
		self.builder.add_edge(self.NODE.CORE, self.NODE.CLEAR_MSGS)
		self.builder.add_edge(self.NODE.CLEAR_MSGS, graphs.END)

		return self.builder.compile(checkpointer=self.memory)

agent_rag = AgentRAG(name="RAG")

2025-03-03 09:10:54.919 | INFO     | toolkit.llm.langchain.execution.agents:build:162 - 🔨 🤖 RAG building
2025-03-03 09:10:54.921 | INFO     | toolkit.llm.langchain.execution.agents:build:164 - 🚀 🤖 RAG built


In [3]:

tests = [
	"What are the main safety procedures to follow before driving the vehicle?",
	"What is the correct driving posture recommended in the manual?",
]

user_input = tests[0]

# async for event in agent_rag.core.astream( 
# 	input={"messages": [("user", user_input)]},
# 	config=agent_rag.config, 
# 	stream_mode="values"
# ):
# 	# event["messages"][-1].pretty_print()
# 	result = event["messages"][-1]
# 	result.pretty_print()

# result = await agent_rag.core.ainvoke(
#   input={"messages": [("user", user_input)]}, config=agent_rag.config, 
# )
# result = await agent_rag.get_agent_result()
# rp_print(result)

## Control

In [7]:

tests = [
	"What's the current fan speed?",
	"Set the fan speed to 3",
	"Set the driver's temperature to 23 degrees",
]
# result = await tool_vehicle_control.execute(tests[0])
# rp_print(result)

In [8]:
from services.llm.agents.vehicle.context import prompts_agent_control
from services.llm.agents.vehicle.tools.control import tool_vehicle_control

class NODE_CTRL(t.EnumCustom):
  SEPARATE_TASKS = t.auto()
  
class DATA_CTRL(t.EnumCustom):
  TASKS = t.auto()

class AgentControl(agents.AgentBase):
	def __init__(
		self, 
		name: str,
		node=NODE_CTRL,
	):
		super().__init__(name=name, llm=inst.llm_main)
		self.node = node
		self.build()
					
	@staticmethod
	def separate_tasks(parsed_query: str) -> list[str]:
		pass

	def node_separate_tasks(
		self,
		state: agents.AgentBase.State,
	) -> agents.AgentBase.State:
		pass

	@utils.print_async_function_name
	async def node_core(
		self, state: agents.AgentBase.State
	) -> agents.AgentBase.State:
		user_query = state['messages'][-1].content
		result = await tool_vehicle_control.execute(user_query)

		return {
			"result": result,
			"messages": [msgs_lc.AIMessage(str(result))],
		}

	def architect(self):
		self.builder.add_node(self.NODE.CORE, self.node_core)

		self.builder.add_edge(graphs.START, self.NODE.CORE)
		self.builder.add_edge(self.NODE.CORE, graphs.END)

		return self.builder.compile(checkpointer=self.memory)

agent_control = AgentControl(name="Control")

2025-03-03 09:11:05.600 | INFO     | toolkit.llm.langchain.execution.agents:build:162 - 🔨 🤖 Control building
2025-03-03 09:11:05.602 | INFO     | toolkit.llm.langchain.execution.agents:build:164 - 🚀 🤖 Control built


In [9]:
user_input = tests[1]

# async for event in agent_control.core.astream(
# 		input={"messages": [("user", user_input)]},
# 		config=agent_control.config, 
#   	stream_mode="values"
# 	):
# 		# event["messages"][-1].pretty_print()
# 		result = event["messages"][-1]
# 		result.pretty_print()

# result = await agent_control.core.ainvoke(
#   input={"messages": [("user", user_input)]}, config=agent_control.config, 
# )
# result = await agent_control.get_agent_result()
# print(result)

## Navigation

In [10]:
from services.llm.agents.vehicle.context import prompts_agent_navigation
from services.llm.agents.vehicle.tools.navigation import tool_navigation

class NODE_NAV(t.EnumCustom):
	pass
  
class DATA_NAV(t.EnumCustom):
	pass

class AgentNavigation(agents.AgentBase):
	def __init__(
    	self, 
     	name: str,
		node=NODE_NAV,
    ):
		super().__init__(name=name, llm=inst.llm_main)
		self.node = node
		self.build()
					
	@utils.print_async_function_name
	async def node_core(
    	self, state: agents.AgentBase.State
    ) -> agents.AgentBase.State:
		user_query = state['messages'][-1].content
		result = await tool_navigation.execute(user_query)

		return {
			"result": result,
			"messages": [msgs_lc.AIMessage(str(result))],
		}

	def architect(self):
		self.builder.add_node(self.NODE.CORE, self.node_core)

		self.builder.add_edge(graphs.START, self.NODE.CORE)
		self.builder.add_edge(self.NODE.CORE, graphs.END)

		return self.builder.compile(checkpointer=self.memory)

agent_navigation = AgentNavigation(name="Navigation")

2025-03-03 09:11:06.455 | INFO     | toolkit.llm.langchain.execution.agents:build:162 - 🔨 🤖 Navigation building
2025-03-03 09:11:06.457 | INFO     | toolkit.llm.langchain.execution.agents:build:164 - 🚀 🤖 Navigation built


In [11]:
tests = [
	"Tìm quán cháo vịt quanh đây",
	"Chỉ đường đến Quán cháo vịt Thanh đa",
	"Chỉ đường đến Quán Ăn Cháo Gỏi Vịt Lái Thiêu",
]

user_input = tests[2]

# async for event in agent_navigation.core.astream( 
# 		input={"messages": [("user", user_input)]},
# 		config=agent_navigation.config, 
#   	stream_mode="values"
# 	):
# 		# event["messages"][-1].pretty_print()
# 		result = event["messages"][-1]
# 		result.pretty_print()

# async for event in agent_navigation.core.astream(
#     input={"messages": [("user", user_input)]},
#     config=agent_navigation.config, 
#     stream_mode="updates",
#     subgraphs=True
# ):
#     graphs.print_graph_event(event)

# result = await agent_navigation.core.ainvoke(
#   input={"messages": [("user", user_input)]}, config=agent_navigation.config, 
# )
# result = await agent_navigation.get_agent_result()
# print(result)

## Music

In [12]:
from services.llm.agents.vehicle.context import prompts_agent_music
from services.llm.agents.vehicle.tools.music import tool_music

class NODE_MUSIC(t.EnumCustom):
	pass
  
class DATA_MUSIC(t.EnumCustom):
	pass

class AgentMusic(agents.AgentBase):
	def __init__(
    	self, 
     	name: str,
		node=NODE_MUSIC,
    ):
		super().__init__(name=name, llm=inst.llm_main)
		self.node = node
		self.build()
					
	@utils.print_async_function_name
	async def node_core(
    	self, state: agents.AgentBase.State
    ) -> agents.AgentBase.State:
		user_query = state['messages'][-1].content
		result = await tool_music.execute(user_query)

		return {
			"result": result,
			"messages": [msgs_lc.AIMessage(str(result))],
		}

	def architect(self):
		self.builder.add_node(self.NODE.CORE, self.node_core)

		self.builder.add_edge(graphs.START, self.NODE.CORE)
		self.builder.add_edge(self.NODE.CORE, graphs.END)

		return self.builder.compile(checkpointer=self.memory)

agent_music = AgentMusic(name="Music")

2025-03-03 09:11:07.194 | INFO     | toolkit.llm.langchain.execution.agents:build:162 - 🔨 🤖 Music building
2025-03-03 09:11:07.195 | INFO     | toolkit.llm.langchain.execution.agents:build:164 - 🚀 🤖 Music built


In [13]:
tests = [
    "Play Shape of You",
    "I want to listen to Hello by Adele",
    "Play some rock music",
    "Put on Bad Guy"
]

user_input = tests[0]

# async for event in agent_music.core.astream( 
# 		input={"messages": [("user", user_input)]},
# 		config=agent_music.config, 
#   	stream_mode="values"
# 	):
# 		# event["messages"][-1].pretty_print()
# 		result = event["messages"][-1]
# 		result.pretty_print()

# async for event in agent_music.core.astream(
#     input={"messages": [("user", user_input)]},
#     config=agent_music.config, 
#     stream_mode="updates",
#     subgraphs=True
# ):
#     graphs.print_graph_event(event)

# result = await agent_music.core.ainvoke(
#   input={"messages": [("user", user_input)]}, config=agent_music.config, 
# )
# result = await agent_music.get_agent_result()
# print(result)

## General

## Main

In [15]:
from services.llm.agents.vehicle.context import prompts_agents_main,  prompts_agents_vehicle

class NODE_MAIN(t.EnumCustom):
	CATEGORIZE_QUERY = t.auto()
	AGENT_RAG = t.auto()
	AGENT_CONTROL = t.auto()
	AGENT_NAVIGATION = t.auto()
	AGENT_MUSIC = t.auto()

class DATA_MAIN(t.EnumCustom):
	RESULT_AGENT_RAG = t.auto()
	RESULT_AGENT_CONTROL = t.auto()
	RESULT_AGENT_NAVIGATION = t.auto()
	RESULT_AGENT_MUSIC = t.auto()

REGISTRY_AGENTS_MAIN: t.Dict[str, agents.AgentBase] = {}
REGISTRY_AGENTS_MAIN[NODE_MAIN.AGENT_RAG] = agent_rag
REGISTRY_AGENTS_MAIN[NODE_MAIN.AGENT_CONTROL] = agent_control
REGISTRY_AGENTS_MAIN[NODE_MAIN.AGENT_NAVIGATION] = agent_navigation
REGISTRY_AGENTS_MAIN[NODE_MAIN.AGENT_MUSIC] = agent_music

T_USER_QUERY_CATEGORY: t.TypeAlias = t.Literal[
  "car_manual", "car_control", "navigation", "general", "music"
]

CATEGORY_2_AGENT: t.Dict[str, str] = {
	"car_manual": NODE_MAIN.AGENT_RAG,
	"car_control": NODE_MAIN.AGENT_CONTROL,
	"navigation": NODE_MAIN.AGENT_NAVIGATION,
	"music": NODE_MAIN.AGENT_MUSIC,
}

class AgentMain(agents.AgentBase):
	def __init__(
		self, 
		name: str,
		node=NODE_MAIN,
	):
		super().__init__(
			name=name, llm=inst.llm_main, registry_agents=REGISTRY_AGENTS_MAIN
		)
		self.node = node
		self.build()
		
	@staticmethod
	@utils.print_async_function_name
	async def categorize_query(user_query: str) -> T_USER_QUERY_CATEGORY:
		result = None
		
		examples = await inst.retrievers_qdrant["user_query_category"].ainvoke(user_query)
		examples = await retrievers.extract_retriever_results(examples)

		prompt = prompts_lc.PromptTemplate.from_template(prompts_agents_vehicle["categorize_query"])
		prompt = await prompt.ainvoke({
			"user_query": user_query,
			"examples": examples,
		})
	
		result = await inst.llm_main.ainvoke(prompt)
		result = result.content

		return result

	@utils.print_async_function_name
	async def node_core(
		self, state: agents.AgentBase.State
	) -> graphs.Command[
			t.Literal[
				NODE_MAIN.AGENT_RAG, NODE_MAIN.AGENT_CONTROL, 
				NODE_MAIN.AGENT_NAVIGATION, NODE_MAIN.AGENT_MUSIC,
				graphs.END,
			]
		]:

		result: agents.AgentBase.State = {}
		
		user_query = state["messages"][-1].content

		latest_agent = state.get("latest_agent", None)

		if latest_agent is None:
			user_query_category = await AgentMain.categorize_query(user_query)
			agent_name = CATEGORY_2_AGENT[user_query_category]

			await clients.namespace_redis_llm.set_value(
				"agent", { "current": agent_name }
			)

			return graphs.Command(
				goto=agent_name,
				update={
					"user_query": user_query,
					"messages": [msgs_lc.AIMessage(str(f"User query category: {user_query_category}"))],
					"latest_agent": agent_name,
				}
			)
		else:
			agent_name = state['latest_agent']
			
			await clients.namespace_redis_llm.set_value(
				"agent", { "current": "manager" }
			)

			child_agent_result = await self.REGISTRY_AGENTS[agent_name].get_agent_result()
			child_agent_msg = await self.REGISTRY_AGENTS[agent_name].format_agent_result()

			prompt_tpl = prompts_lc.PromptTemplate.from_template(prompts_agents_main["system"])
			
			prompt = await prompt_tpl.ainvoke({
				"user_input": user_query,
				"child_agent_result": child_agent_result,
			})

			result: msgs_lc.AIMessage = await self.llm.ainvoke(prompt)
			
			final_msg = result
   
			# final_msg = msgs_lc.AIMessage(f"Final result:\n{child_agent_msg.content}")
   
		return graphs.Command(
				goto=graphs.END,
				update={
					"result": final_msg.content,
					"messages": [final_msg],
					"latest_agent": None,
					"user_query": None,
					"user_query_org": None,
					"user_query_category": None,
				}
			)
	
	def architect(self):
		
		self.builder.add_node(self.NODE.CORE, self.node_core)
		self.builder.add_node(self.node.AGENT_RAG, self.node_agent_rag)
		self.builder.add_node(self.node.AGENT_CONTROL, self.node_agent_control)
		self.builder.add_node(self.node.AGENT_NAVIGATION, self.node_agent_navigation)
		self.builder.add_node(self.node.AGENT_MUSIC, self.node_agent_music)

		self.builder.add_edge(graphs.START, self.NODE.CORE)

		return self.builder.compile(checkpointer=self.memory)
	
	@utils.print_async_function_name
	async def node_agent_rag(
		self,
		state: agents.AgentBase.State,
	) -> graphs.Command[t.Literal[AgentMain.NODE.CORE]]:

		result = await agent_rag.core.ainvoke(
			input={"messages": [("user", state["user_query"])]}, config=agent_rag.config,
		)
		result = await agent_rag.get_agent_result()
		msg = await agent_rag.format_agent_result()
	
		return graphs.Command(
			goto=AgentMain.NODE.CORE,
			update={
				"data": {
					DATA_MAIN.RESULT_AGENT_RAG: result,
				},
				"messages": [msg],
			}
		)

	@utils.print_async_function_name
	async def node_agent_control(
		self,
		state: agents.AgentBase.State,
	) -> graphs.Command[t.Literal[AgentMain.NODE.CORE]]:

		result = await agent_control.core.ainvoke(
			input={"messages": [("user", state["user_query"])]}, config=agent_control.config
		)
		result = await agent_control.get_agent_result()
		msg = await agent_control.format_agent_result()

		return graphs.Command(
			goto=AgentMain.NODE.CORE,
			update={
				"data": {
					DATA_MAIN.RESULT_AGENT_CONTROL: result,
				},
				"messages": [msg],
			}
		)

	@utils.print_async_function_name
	async def node_agent_navigation(
		self,
		state: agents.AgentBase.State,
	) -> graphs.Command[t.Literal[AgentMain.NODE.CORE]]:

		result = await agent_navigation.core.ainvoke(
			input={"messages": [("user", state["user_query"])]}, config=agent_navigation.config
		)
		result = await agent_navigation.get_agent_result()
		msg = await agent_navigation.format_agent_result()

		return graphs.Command(
			goto=AgentMain.NODE.CORE,
			update={
				"data": {
					DATA_MAIN.RESULT_AGENT_NAVIGATION: result,
				},
				"messages": [msg],
			}
		)

	@utils.print_async_function_name
	async def node_agent_music(
		self,
		state: agents.AgentBase.State,
	) -> graphs.Command[t.Literal[AgentMain.NODE.CORE]]:

		result = await agent_music.core.ainvoke(
			input={"messages": [("user", state["user_query"])]}, config=agent_music.config
		)
		result = await agent_music.get_agent_result()
		msg = await agent_music.format_agent_result()

		return graphs.Command(
			goto=AgentMain.NODE.CORE,
			update={
				"data": {
					DATA_MAIN.RESULT_AGENT_MUSIC: result,
				},
				"messages": [msg],
			}
		)

agent_main = AgentMain(name="Main")

2025-03-03 09:11:28.341 | INFO     | toolkit.llm.langchain.execution.agents:build:162 - 🔨 🤖 Main building
2025-03-03 09:11:28.359 | INFO     | toolkit.llm.langchain.execution.agents:build:164 - 🚀 🤖 Main built


In [ ]:
tests = [
	"What are the main safety procedures to follow before driving the vehicle?",
    "It's too hot",
	"What's the current fan speed?",
	"Set the fan speed to 3",
	"Find nearby HighLand Coffee",
    "Give direction to Ly Thuong Kiet HighLand coffe",
	"Play Shape of You",
	"Stop the music",
    "Continue playing music",
]

user_input = tests[8]
	
async for event in agent_main.core.astream(
    input={"messages": [("user", user_input)]},
    config=agent_main.config, 
    stream_mode="updates",
    subgraphs=True
):
    graphs.print_graph_event(event)

# events = []
# async for event in agent_main.core.astream(
# 	input={"messages": [("user", user_input)]},
# 	config=agent_main.config, 
# 	stream_mode="values",
# ):
# 	event["messages"][-1].pretty_print()
# 	result = event["messages"][-1]
# 	# result.pretty_print()
# 	events.append(event)

# result = await agent_main.core.ainvoke(
#   input={"messages": [("user", user_input)]}, config=agent_main.config, 
# )
# result = await agent_main.get_agent_result()
# print(result)

2025-03-03 09:14:35.470 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async AgentMain.node_core
2025-03-03 09:14:35.471 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async str.categorize_query
2025-03-03 09:14:36.919 | INFO     | toolkit.db.redix:set_value:72 - Setting {'current': agent_music} for key: llm:agent
2025-03-03 09:14:36.920 | INFO     | toolkit.db.redix:set_value:79 - Successfully set {"current": "agent_music"} for key: llm:agent


╭───────────────────────────────────────────────────── core ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─ AI Message ────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ User query category: music                                                                                  │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-03-03 09:14:36.923 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async AgentMain.node_agent_music
2025-03-03 09:14:36.925 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async AgentMusic.node_core
2025-03-03 09:14:36.925 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async ToolMusic.execute
2025-03-03 09:14:36.925 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async ToolMusic.parse_user_query
2025-03-03 09:14:37.502 | INFO     | services.llm.agents.vehicle.tools.music:parse_user_query:519 - Parsed music query result: {'action': 'continue'}
2025-03-03 09:14:37.503 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async ToolMusic._continue_music
2025-03-03 09:14:37.503 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async ToolMusic.get_current_track
2025-03-03 09:14:37.508 | INFO     | services.llm.agents.vehicle.tools.music:_continue_music:448 - Set continue play to true for track: Ed Sheeran - Shape of You (Official Music Vid

╭────────────────────────────────────────────────── agent_music ──────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─ AI Message ────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Agent Music's result:                                                                                       │ │
│ │ {'success': True, 'action': 'continue', 'data': {'continued_track': 'Ed Sheeran - Shape of You (Official    │ │
│ │ Music Video)'}}                                                                                             │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

2025-03-03 09:14:37.513 | INFO     | toolkit.utils.utils:wrapper:420 - 📟 Calling async AgentMain.node_core
2025-03-03 09:14:37.513 | INFO     | toolkit.db.redix:set_value:72 - Setting {'current': 'manager'} for key: llm:agent
2025-03-03 09:14:37.515 | INFO     | toolkit.db.redix:set_value:79 - Successfully set {"current": "manager"} for key: llm:agent


╭───────────────────────────────────────────────────── core ──────────────────────────────────────────────────────╮
│                                                                                                                 │
│ ╭─ AI Message ────────────────────────────────────────────────────────────────────────────────────────────────╮ │
│ │ Shape of You by Ed Sheeran has been selected and will begin playing.                                        │ │
│ ╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────╯ │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

In [ ]:
result = await agent_main.categorize_query(tests[7])
print(result)

In [ ]:
from IPython.display import display, Image

display(Image(agent_main.core.get_graph().draw_mermaid_png()))

## Template

# Ref

In [ ]:
prompt_tpl = prompts_lc.ChatPromptTemplate([
	("system", prompts_agents_rag["system"]),
	("human", prompts_agents_rag["generate"]),
])

prompt = await prompt_tpl.ainvoke({
	"user_query": user_query,
	"retrieved_data": state["data"][DATA_RAG.RETRIEVED_DATA],
})

result = await inst.llm_main.ainvoke(prompt)

In [ ]:
prompt_tpl = prompts_lc.PromptTemplate.from_template(prompts_agents_vehicle["categorize_query"])
prompt = await prompt_tpl.ainvoke({
	"user_query": user_query,
	"examples": examples,
})

result = await inst.llm_main.ainvoke(prompt)